In [13]:
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from scipy import stats
warnings.filterwarnings("ignore")
from lightgbm import LGBMClassifier
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, Normalizer
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [6]:
data = pd.read_csv('carprices.csv', sep=';', encoding='ISO-8859-1')

In [7]:
data = data[['model', 'transmission', 'fuelType', "productionDate",'milage','priceEuro']]
data = data.rename(columns={"productionDate": "year", "priceEuro":"price"})

In [10]:
data.isnull().sum()

model           0
transmission    0
fuelType        0
year            0
milage          0
price           0
dtype: int64

In [9]:
print("Shape of train data Before dropping any Row: ",data.shape)
data = data[data['transmission'].notna()]
print("Shape of train data After dropping Rows with NULL values in Mileage: ",data.shape)
data = data[data['fuelType'].notna()]
print("Shape of train data After dropping Rows with NULL values in Engine : ",data.shape)
data = data.reset_index(drop=True)
print(data.shape)

Shape of train data Before dropping any Row:  (155178, 6)
Shape of train data After dropping Rows with NULL values in Mileage:  (133044, 6)
Shape of train data After dropping Rows with NULL values in Engine :  (129321, 6)
(129321, 6)


In [14]:
test = data[['milage',"price"]]
z = np.abs(stats.zscore(test))
outliers = np.where(z > 3)
data = data.drop(data.index[outliers[0]])
data = data.reset_index(drop=True)
print(data.shape)

(125874, 6)


In [15]:
data.fuelType.value_counts()

Diesel      70650
Petrol      45886
Hybrid       4683
Electric     4588
CNG            37
LPG            29
Ethanol         1
Name: fuelType, dtype: int64

In [16]:
data = data[data['fuelType'] != "Ethanol"].reset_index(drop=True)
data = data[data['fuelType'] != "LPG"].reset_index(drop=True)
data = data[data['fuelType'] != "CNG"].reset_index(drop=True)
print(data.shape)

(125807, 6)


In [17]:
data['model'].value_counts()

Volkswagen Golf 1.6 TDI                  1620
Audi A4 2.0 TDI                          1168
Mercedes Benz C-Class                    1110
Bmw 3 Series 320d                         978
Volkswagen Passat 2.0 TDI                 932
                                         ... 
Ds Automobiles Ds 3 Crossback electro       1
Jeep Wrangler 3.8 Sahara                    1
Peugeot 208 1.6 HDi                         1
Mitsubishi Pajero 1.8                       1
Nissan Micra 1.6                            1
Name: model, Length: 3861, dtype: int64

In [18]:
counts = data['model'].value_counts()
to_keep = counts[counts > 10].index
print(data.shape)
data = data[data['model'].isin(to_keep)]
print(data.shape)

(125807, 6)
(117242, 6)


In [19]:
train_data = data

In [20]:
X_train, X_test, y_train, y_test = train_test_split(train_data[['model','transmission','fuelType', 'year','milage']],train_data.price,test_size = 0.2, random_state=42)

In [21]:
listtrain = X_train['model']
listtest = X_test['model']
  
# prints the missing in listrain 
print("Missing values in first list:", (set(listtest).difference(listtrain))) 

Missing values in first list: set()


In [22]:
check_result = pd.concat([X_test, y_test], axis=1, join="inner")

In [23]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
le = LabelEncoder()
X_train['model'] = le.fit_transform(X_train['model'])
X_train['transmission'] = le.fit_transform(X_train['transmission'])
X_train['fuelType'] = le.fit_transform(X_train['fuelType'])
le1 = LabelEncoder()
X_test['model'] = le1.fit_transform(X_test['model'])
X_test['transmission'] = le1.fit_transform(X_test['transmission'])
X_test['fuelType'] = le1.fit_transform(X_test['fuelType'])

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler()
X_train[['year', 'milage']] = scaler1.fit_transform(X_train[['year', 'milage']])

scaler2 = MinMaxScaler()
X_test[['year', 'milage']] = scaler2.fit_transform(X_test[['year', 'milage']])

In [26]:
scores = cross_val_score(RandomForestRegressor(n_estimators= 10000,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=25), X_train, y_train, cv=5)
print("Cross-validation scores: {}".format(scores))
print("Cross-validation mean score: {:.2f}".format(scores.mean()))

,model,transmission,fuelType,year,milage
65811,262,0,0,0.949580,0.496359
72096,139,0,0,0.983193,0.071954
67947,237,0,2,0.991597,0.039280
112885,24,1,3,0.991597,0.000014
124785,1310,1,0,0.949580,0.526916
...,...,...,...,...,...
122876,1203,1,3,0.983193,0.023033
62168,316,0,3,0.983193,0.068248
45755,613,1,0,0.983193,0.000036
124814,1198,1,0,0.941176,0.254250


In [29]:
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_absolute_error

Acc = pd.DataFrame(columns=['model', 'Cross-validation mean score', 'Accuracy on Testing set', 'MAE'])
regressors = [['DecisionTreeRegressor', DecisionTreeRegressor()],
              ['XGBRegressor', xgb.XGBRegressor(n_estimators=10000, learning_rate=0.06)],
              ['RandomForestRegressor', RandomForestRegressor(n_estimators=10000,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=25)],
              ['ExtraTreesRegressor',ExtraTreesRegressor(n_estimators=10000,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=25)]]

for mod in regressors:
    name = mod[0]
    model = mod[1]
    cvc = cross_val_score(model, X_train, y_train, cv=5)
    cvp = cross_val_predict(model, X_test, y_test, cv=5)
    mae = mean_absolute_error(y_test, cvp)
    
    Acc = Acc.append(pd.Series({'model':name, 'Cross-validation mean score':cvc.mean(), 'Accuracy on Testing set': cvp.mean(), 'MAE': mae}), ignore_index=True)


In [39]:
Acc

,model,Cross-validation mean score,Accuracy on Testing set,MAE
0,DecisionTreeRegressor,0.906462,32955.129016,5370.176623
1,XGBRegressor,0.945013,32930.957031,3971.919761
2,RandomForestRegressor,0.911661,32886.468402,5288.260306
3,ExtraTreesRegressor,0.908358,32945.518739,5293.603730


In [35]:
from sklearn.metrics import r2_score
y_pred = cross_val_predict(xgb.XGBRegressor(n_estimators=10000, learning_rate=0.06), X_test, y_test, cv=5)
submission = pd.DataFrame({'Car_id': check_result.index,'model': check_result.model, 'year': check_result.year, 'predicted_price':y_pred, 'actual_price': y_test}) 
r2 = r2_score(y_test, y_pred)
print("R-squared: ", r2)

R-squared:  0.8887011131038449


,Car_id,model,year,predicted_price,actual_price
65811,65811,Bmw 5 Series 530d,2017,38092.882812,37900
72096,72096,Audi Q3 2.0,2021,51568.355469,51780
67947,67947,Bmw 3 Series 330d,2022,65287.792969,59479
112885,112885,Audi A3,2022,36177.648438,29410
124785,124785,Volkswagen Touran,2017,18720.476562,17951
87873,87873,Mercedes Benz C-Class 43 AMG,2017,21471.546875,16490
117760,117760,Volkswagen Polo 1.0 TSI DSG,2022,27855.591797,28790
19126,19126,Toyota C-Hr,2022,32645.695312,37951
46086,46086,Jaguar I-Pace,2022,77407.460938,80861
78127,78127,Audi A3 2.0 TDI,2022,46669.925781,46469


In [38]:
submission.head(10)

,Car_id,model,year,predicted_price,actual_price
65811,65811,Bmw 5 Series 530d,2017,38092.882812,37900
72096,72096,Audi Q3 2.0,2021,51568.355469,51780
67947,67947,Bmw 3 Series 330d,2022,65287.792969,59479
112885,112885,Audi A3,2022,36177.648438,29410
124785,124785,Volkswagen Touran,2017,18720.476562,17951
87873,87873,Mercedes Benz C-Class 43 AMG,2017,21471.546875,16490
117760,117760,Volkswagen Polo 1.0 TSI DSG,2022,27855.591797,28790
19126,19126,Toyota C-Hr,2022,32645.695312,37951
46086,46086,Jaguar I-Pace,2022,77407.460938,80861
78127,78127,Audi A3 2.0 TDI,2022,46669.925781,46469


In [37]:
mean_absolute_error(y_test, y_pred)

3971.919761098419

In [ ]:
y_predicted, y_actual = Models(GradientBoostingRegressor(random_state=21, n_estimators=3000))
Graph_prediction(150, y_actual, y_predicted)

In [40]:
cvc_gbr = cross_val_score(GradientBoostingRegressor(random_state=21, n_estimators=10000), X_train, y_train, cv=5)
print(cvc_gbr.mean())

0.9217229035027621


In [41]:
y_pred1 = cross_val_predict(GradientBoostingRegressor(random_state=21, n_estimators=10000), X_test, y_test, cv=5)
submission2 = pd.DataFrame({'Car_id': check_result.index,'model': check_result.model, 'year': check_result.year, 'predicted_price':y_pred1, 'actual_price': y_test}) 
r2 = r2_score(y_test, y_pred1)
print("R-squared: ", r2)

R-squared:  0.8926601139151453


In [45]:
submission2.tail(30)

,Car_id,model,year,predicted_price,actual_price
105126,105126,Audi Q8,2020,99237.279179,90042
13627,13627,Suzuki Baleno 1.2,2018,19936.183804,13500
88899,88899,Mercedes Benz Gle-Class 350 4MATIC,2017,64981.740804,64800
116633,116633,Volkswagen Touran 1.6 TDI,2017,19854.540740,17490
45659,45659,Jaguar Xf,2017,25635.749363,26903
52485,52485,Ford Focus 1.5,2020,17038.986080,17490
28214,28214,Opel Corsa 1.2 Turbo,2021,15239.968214,17490
11440,11440,Volkswagen Golf 1.0 tsi,2017,14925.348632,8500
63675,63675,Bmw 5 Series 530e xDrive,2022,76719.394790,60339
70675,70675,Audi Q5 2.0,2019,49236.179248,56490


In [44]:
mean_absolute_error(y_test, y_pred1)

4183.4887614632125